<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MYBERT_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Dec 21 18:29:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   32C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install flash-attn -q
!pip install --upgrade transformers -q # Upgrade to the latest version of transformers
!pip install git+https://github.com/huggingface/transformers.git -q
!pip install git+https://github.com/answerdotai/ModernBERT.git # Install ModernBERT from the source
!pip install triton -q # Install the triton library

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

model_id = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model on the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForMaskedLM.from_pretrained(model_id, trust_remote_code=True).to(device)

In [5]:
def get_prediction(text):
  # Add the [MASK] token at the end for this specific case
  if "[MASK]" not in text:
    text = text + " is [MASK]."
  inputs = tokenizer(text, return_tensors="pt").to(device)
  outputs = model(**inputs)
  masked_index = inputs["input_ids"][0].tolist().index(tokenizer.mask_token_id)
  predicted_token_id = outputs.logits[0, masked_index].argmax(axis=-1)
  predicted_token = tokenizer.decode(predicted_token_id)
  return predicted_token

In [6]:
query= "The capital of Canada?"
response=str(get_prediction(query))
print('Question: %s' % query)
print("  Answer:%s" %response)

Question: The capital of Canada?
  Answer: Ottawa


In [7]:
query= "The capital of Russia?"
response=str(get_prediction(query))
print('Question: %s' % query)
print("  Answer:%s" % response)

Question: The capital of Russia?
  Answer: Moscow
